## 이터레이터와 제네레이터

In [4]:
# 간단한 제네레이터 함수
def count_up_to(max):
    count = 1
    while count < max:
        yield count # 값을 생성하고 함수 실행 일시 중지
        count += 1
        
counter = count_up_to(5)
print(f"제네레이터 출력: {next(counter)}")
print(f"제네레이터 출력: {next(counter)}")
print(f"제네레이터 출력: {next(counter)}")
print(f"제네레이터 출력: {next(counter)}")


for num in count_up_to(3):
    print(f"for loop 출력: {num}")

제네레이터 출력: 1
제네레이터 출력: 2
제네레이터 출력: 3
제네레이터 출력: 4
for loop 출력: 1
for loop 출력: 2


In [6]:
# 리스트 컴프리헨션 vs 제네레이터 표현식
sq_list = [x ** 2 for x in range(10)] # list comprehension
sq_gen = (x**2 for x in range(10)) # generator 

print(sq_list)
print(sq_gen)

for num in sq_gen:
    print(num)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
<generator object <genexpr> at 0x11559d560>
0
1
4
9
16
25
36
49
64
81


In [9]:
# 제네레이터 함수의 상태 관리

def stat_gen():
    print("first val")
    yield 1
    
    print("second val")
    yield 2
    
    print("third val")
    yield 3
    
gen = stat_gen()
print(next(gen))
print("middle task process")
print(next(gen))
print(next(gen))

first val
1
middle task process
second val
2
third val
3


In [14]:
# 센서 -> 데이터를 보냄(실시간 데이터 스트리밍 시뮬레이션)
import time
import random

def sensor_data_stream():
    while True:
        temp = 20 + random.uniform(-5,5)
        yield f"온도 : {temp:.2f}C, 시간 : {time.strftime('%H:%M:%S')}"
    
    
stream = sensor_data_stream()
for _ in range(5):
    print(next(stream))

온도 : 18.91C, 시간 : 09:25:16
온도 : 18.57C, 시간 : 09:25:16
온도 : 16.54C, 시간 : 09:25:16
온도 : 19.24C, 시간 : 09:25:16
온도 : 17.34C, 시간 : 09:25:16


## 동시성과 병렬처리

In [30]:
import threading
import time

stop_event = threading.Event()

def background_task():
    while not stop_event.is_set():
        print("backgound process progressing...")
        time.sleep(1)

#making daemon thread
daemon_thread = threading.Thread(target = background_task,daemon = True)
daemon_thread.start()

print("main thread processing")
time.sleep(3)

stop_event.set()
print("main thread finished")

backgound process progressing...main thread processing

backgound process progressing...
backgound process progressing...
main thread finished


In [32]:
# 스레드 동기화 도구
import threading
import time

event = threading.Event()

def waiter():
    print("대기자: 이벤트를 기다리는 중")
    event.wait()
    print("대기자: 이벤트를 수신하고 작업 진행!")          
    
def setter():
    print("설정자: 작업 중...")
    time.sleep(3)
    print("설정자:이제 이벤트를 설정합니다.")
    event.set()
    
    
# 스레드 시작
t1 = threading.Thread(target = waiter)
t2 = threading.Thread(target = setter)

t1.start()
t2.start()

대기자: 이벤트를 기다리는 중
설정자: 작업 중...


설정자:이제 이벤트를 설정합니다.
대기자: 이벤트를 수신하고 작업 진행!


In [37]:
## condition

import threading
import time

data = None
condition = threading.Condition()

def wait_for_data():
    print("대기 스레드 데이터를 기다립니다...")
    
    with condition:
        condition.wait()
        
        print(f"대기 스레드: 데이터 {data}를 받았습니다!")
    
def prepare_data():
    global data
    
    print("준비 스레드: 데이터 준비 중...")
    time.sleep(2)
    
    with condition:
        data = "준비된 데이터"
        print("준비 스레드:데이터가 준비 되었습니다. ")
        condition.notify()
        
        
t1 = threading.Thread(target = wait_for_data)
t2 = threading.Thread(target = prepare_data)

t1.start()
t2.start()

t1.join()
t2.join()

대기 스레드 데이터를 기다립니다...
준비 스레드: 데이터 준비 중...
준비 스레드:데이터가 준비 되었습니다. 
대기 스레드: 데이터 준비된 데이터를 받았습니다!


In [42]:
# 작업 분배 시스템

import threading
import queue
import time
import random

task_queue = queue.Queue()
result_queue = queue.Queue()

def create_tasks():
    print("작업 생성 시작")
    #10개의 작업 생성
    for i in range(10):
        task = f"작업-{i}"
        task_queue.put(task)
        print(f"작업 추가 : {task}")
        time.sleep(random.uniform(0.1,0.3)) # 약간의 간격을 두고 작업 생성
        
    # 영업 종료 신호(워커 수만큼), 워커 쓰레드 생성
    for _ in range(3):
        task_queue.put(None)
    print("모든 작업 생성 완료")
    
# 작업 처리 함수
def worker(worker_id):
    print(f"워커 {worker_id}시작") #  출근 신고
    while True: #계속 일하는 무한 루프
        #작업 가져오기 (주문서 확인)
        task = task_queue.get() #  보관함에서 작업 꺼냄
        
        #퇴근 시간인지 확인
        if task is None:
            print(f"워커 {worker_id} 종료") #퇴근!
            break
        
        #작업 처리(요리 만들기)
        print(f"워커 {worker_id}가 {task} 처리 중...")
        processing_time = random.uniform(0.5,1.5) # 요리 시간은 랜덤
        time.sleep(processing_time)
        
        #  결과 제출 (완성된 요리 올려두기)
        result = f"{task} 완료 (소요시간 : {processing_time:.2f}초)"
        result_queue.put((worker_id, result))
        
        # 작업 완료 표시 (주문서에 완료 도장)
        task_queue.task_done() # 현재 처리 중인 특정 작업 하나가 완료, 전체 작업이 끝났다는 의미는 아님
        print(f"남은 작업 수: {task_queue.qsize()}")
        
def result_collector():
    print("결과 수집기 시작")
    results = []
    
    for _ in range(10):
        worker_id, result = result_queue.get()
        print(f"결과 수신 워커: {worker_id}-> {result}")
        result.append(result)
        result_queue.task_done()
    
    print(f"총 {len(results)}개 결과 수집 완료")
    
    
creator = threading.Thread(target= create_tasks)
workers = [threading.Thread(target = worker, args = (i,))for i in range(3)]
collector = threading.Thread(target = result_collector)

creator.start()
for w in workers:
    w.start()
collector.start()

creator.join()
for w in workers:
    w.join()
collector.join()

print("모든 작업 완료!")


작업 생성 시작
작업 추가 : 작업-0
워커 0시작
워커 0가 작업-0 처리 중...
워커 1시작
워커 2시작
결과 수집기 시작
작업 추가 : 작업-1워커 1가 작업-1 처리 중...

작업 추가 : 작업-2
워커 2가 작업-2 처리 중...
작업 추가 : 작업-3


Exception in thread Thread-368 (result_collector):
Traceback (most recent call last):
  File "/Users/apple/.pyenv/versions/3.12.11/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/Users/apple/.pyenv/versions/3.12.11/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "/Users/apple/.pyenv/versions/3.12.11/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/hs/81ncphtd2yzggndhp3qhjq400000gn/T/ipykernel_1282/1817078855.py", line 57, in result_collector
AttributeError: 'str' object has no attribute 'append'


남은 작업 수: 1결과 수신 워커: 0-> 작업-0 완료 (소요시간 : 0.76초)

워커 0가 작업-3 처리 중...
작업 추가 : 작업-4
작업 추가 : 작업-5
작업 추가 : 작업-6
작업 추가 : 작업-7
남은 작업 수: 4
워커 2가 작업-4 처리 중...
남은 작업 수: 3
워커 1가 작업-5 처리 중...
작업 추가 : 작업-8
남은 작업 수: 3
워커 0가 작업-6 처리 중...
작업 추가 : 작업-9
모든 작업 생성 완료
남은 작업 수: 6
워커 2가 작업-7 처리 중...
남은 작업 수: 5
워커 1가 작업-8 처리 중...
남은 작업 수: 4
워커 0가 작업-9 처리 중...
남은 작업 수: 3
워커 2 종료
남은 작업 수: 2
워커 0 종료
남은 작업 수: 1
워커 1 종료
모든 작업 완료!


In [51]:
# 비동기 프로그래밍

import asyncio

#코루틴 정의

async def say_hello(name, delay):
    print(f"{name} start greeting")
    await asyncio.sleep(delay) # 비동기적으로 대기
    print(f"{name} finish greeting (delay time: {delay}s)")
    return f"{name}'s result"


async def main():
    print("start program")

    results = await asyncio.gather(
        say_hello("A",3),
        say_hello("B",1),
        say_hello("C",2),
    )
    
    print(f"all results: {results}")
    print("프로그램 종료")
    
    # 이벤트 루프 생성 및 실행
    if __name__ == "__main__":
        asyncio.run(main())

In [54]:
!pip install aiohttp

15355.15s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [58]:
# 여러 웹사이트에서 동시에 정보 수집
import asyncio
import aiohttp
import time

# 대상 웹사이트 목록
websites = [
    "https://www.google.com",
    "https://www.naver.com",
    "https://www.daum.net",
    "https://www.github.com",
    "https://www.python.org"
]

# 비동기적으로 웹사이트 내용 가져오기
async def fetch(session,url):
    print(f"{url}요청시작")
    try:
        start_time = time.time()
        async with session.get(url, timeout = 10) as respone:
            content = await response.text()
            elapsed = time.time() -start_time
            print(f"{url} 응답 완료: {len(content)} 바이트 (소요시간: {elapsed:.2f}s)")
            return url, len(content), elapsed
    except Exception as e:
        print(f"{url} 오류 발생 : {e}")
        return url, 0,0
    
    
# 모든 웹사이트 순차적으로 요청
async def fetch_all_sequential(urls):
    start_time = time.time()
    results = []


    async with aiohttp.ClientSession() as session:
        for url in urls:
            result = await fetch(session, url)            
            results.append(result)
    end_time = time.time()
    print(f"순차 처리 완료: {end_time- start_time:.2f}s 소요")
    return results

    async def fetch_all_parallel(urls):
        start_time = time.time()
        
        async with aiohttp.ClientSession() as session:
            tasks = [fetch(session, url) for url in urls]
            results = await asyncio.gather(*tasks)  # ✅ 병렬 실행
        print(f"병렬 처리 완료: {time.time() - start_time:.2f}s 소요")
        return results

# 메인 함수
async def main():
    print("\n=====순차 처리 시작=======")
    sequential_results = await fetch_all_sequential(websites)
    
    #잠시 대기
    await asyncio.sleep(1)
    
    #병렬 처리
    print("\n=====병렬 처리 시작=======")
    parallel_results = await fetch_all_parallel(websites)
    
    #결과 요약
    print('\n=====결과 요약 =====')
    seq_total_bytes = sum(r[1] for r in sequential_results)
    par_total_bytes = sum(r[1] for r in parallel_results)
    
    print(f"순차 처리 : 총 {seq_total_bytes}bytes")
    print(f"병렬 처리 : 총 {par_total_bytes}bytes")
    
if __name__ == "__main__":
    asyncio.run(main())
    

/Users/apple/.pyenv/versions/3.12.11/lib/python3.12/ast.py:52: RuntimeWarning: coroutine 'main' was never awaited
  return compile(source, filename, mode, flags,


RuntimeError: asyncio.run() cannot be called from a running event loop

## 데이터 사이언스

In [59]:
import numpy as np

ModuleNotFoundError: No module named 'numpy'

In [60]:
import numpy as np

ModuleNotFoundError: No module named 'numpy'